In [4]:
# usamos el kernel interoperability_R_python
import anndata
import numpy
import scanpy
import mudata
import tempfile
import os
import rpy2
import rpy2.robjects as ro
import anndata2ri
from scipy.sparse import csr_matrix

# Crear un conversor local para anndata2ri
converter = anndata2ri.converter

# Registrar el conversor local
with ro.conversion.localconverter(converter):
    # Aquí irían los comandos que requieren la conversión entre R y Python
    pass

# Cargar la extensión rpy2 para usar comandos R en Python
%reload_ext rpy2.ipython


In [2]:
# Create a randomly generated AnnData object to use as an example
counts = csr_matrix(numpy.random.poisson(1, size=(100, 2000)), dtype=numpy.float32)
adata = anndata.AnnData(counts)
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
# Do some standard processing to populate the object
scanpy.pp.calculate_qc_metrics(adata, inplace=True)
adata.layers["counts"] = adata.X.copy()
scanpy.pp.normalize_total(adata, inplace=True)
scanpy.pp.log1p(adata)
scanpy.pp.highly_variable_genes(adata, inplace=True)
scanpy.tl.pca(adata)
scanpy.pp.neighbors(adata)
scanpy.tl.umap(adata)
adata

/home/evercher/miniforge3/envs/interoperability_R_python/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'

In [3]:
temp_dir = tempfile.TemporaryDirectory()
h5ad_file = os.path.join(temp_dir.name, "example.h5ad")

adata.write_h5ad(h5ad_file)

In [6]:
%%R
library(Seurat)
library(SeuratDisk)

R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [ ]:
%%R -i h5ad_file

#Esta línea es una "magic command" de Jupyter que importa la variable h5ad_file de Python a R.

message("Converting H5AD to H5Seurat...")
SeuratDisk::Convert(h5ad_file, dest = "h5seurat", overwrite = TRUE)
message("Reading H5Seurat...")
h5seurat_file <- gsub(".h5ad", ".h5seurat", h5ad_file)
seurat <- SeuratDisk::LoadH5Seurat(h5seurat_file, assays = "RNA")
message("Read Seurat object:")
seurat


In [20]:
# Convertir la matriz dispersa a una matriz densa de NumPy
counts_mat_dense = adata.layers["counts"].T.toarray()


In [21]:
%%R -i counts_mat_dense -o magic_cpm
library(scuttle)
# Calcular CPM en R
magic_cpm <- scuttle::calculateCPM(counts_mat_dense)


R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC

In [28]:
counts_mat = adata.layers["counts"].T

In [29]:
%%R -i expression_matrix
library(scuttle)
qc <- scuttle::perCellQCMetrics(as.matrix(expression_matrix))
head(qc)


DataFrame with 6 rows and 2 columns
        sum  detected
  <numeric> <numeric>
1   57.5238        64
2   56.7669        60
3   58.4022        67
4   64.7061        70
5   54.0033        62
6   53.6583        58
